# Beginner's Guide to XGBoost for Classification Problems
## Utilize the hottest ML library for state-of-the-art performance
<img src='images/pexels.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@dom-gould-105501?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Dom Gould</a>
        on 
        <a href='https://www.pexels.com/photo/rear-view-of-silhouette-man-against-sky-during-sunset-325790/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Setup

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb

### What is XGBoost and why is it so popular?

Let me introduce you to the hottest Machine Learning library in the ML community - XGBoost. In recent years, it has been the main driving force behind the algorithms that win massive ML competitions. Its speed and performance is unparalleled and it consistently outperforms any other algorithms aimed at supervised learning tasks.

The library is parallelizable which means the core algorithm can run on clusters of GPUs or even across a network of computers. This makes it feasible to solve ML tasks by training on hundreds of millions of training examples with high performance. 

Originally, it was written in C++ as a command line application. After winning a huge competition in the field of physics, it started being widely adopted by the ML community. As a result, now the library has its APIs in several other languages including Python, R and Julia. 

In this post, we will learn the fundamentals of XGBoost by solving classification tasks. ## TODO

### Refresher on Terminology

Before we move on to code examples of XGBoost, let's refresh on some of the terms we will be using throughout the post. 

**Classification task**: a supervised machine learning task in which one should predict if an instance is in some category by studying the item's features. For example, by looking at the body measurements, patient history and glucose levels of a person, you can predict whether a person belongs to 'Has diabetes' or 'Does not have diabetes' group.

**Binary classification**: One type of classification where the target instance can only belong to either one of two classes. For example, predicting whether an email is spam or not, whether a customer purchases some product or not, etc.

**Multi-class classification**: Another type of classification problems where the target can belong to one of many categories. For example, predicting the species of a bird, guessing someones bloody type, etc.

If you find yourself confused by other terminology, I have written a small ML dictionary for beginners:

https://towardsdatascience.com/codeless-machine-learning-dictionary-for-dummies-fa912cc7bdfe

### An Example of XGBoost For a Classification Problem

### What Powers XGBoost Under the Hood

### Cross-validation OR Hyperparameter Tuning